In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

# 1. LOAD DATA

In [30]:
bureau_df = pd.read_csv(r'../data/raw/bureau.csv')
bureau_bl = pd.read_csv(r'../data/raw/bureau_balance.csv')
application_train = pd.read_csv(r'../data/raw/application_train.csv', usecols = ['SK_ID_CURR', 'TARGET'])
application_test = pd.read_csv(r'../data/raw/application_test.csv', usecols = ['SK_ID_CURR'])

In [31]:
bureau_df.shape

(1716428, 17)

In [32]:
bureau_df.drop(columns='AMT_CREDIT_MAX_OVERDUE', inplace=True)
bureau_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [33]:
bureau_bl.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


# 2. OUTLINERS & MISSING VALUES

In [34]:
outlier_cols = ["DAYS_CREDIT_ENDDATE", "DAYS_CREDIT_UPDATE", "DAYS_ENDDATE_FACT"]

for col in outlier_cols:
    bureau_df.loc[bureau_df[col] < -20000, col] = np.nan

## 2.1 BUREAU_BL

In [ ]:
#  Encode STATUS theo ordinal mapping
status_map = {
    'C': 0,   # closed loan 
    '0': 0,   # paid on time
    'X': 1,   # unknown → neutral/light risk
    '1': 2,
    '2': 3,
    '3': 4,
    '4': 5,
    '5': 6    
}
bureau_bl["STATUS_NUM"] = bureau_bl["STATUS"].map(status_map)

# 2.2 Aggregate bureau_balance theo SK_ID_BUREAU
bb_agg = bureau_bl.groupby("SK_ID_BUREAU").agg({
    "MONTHS_BALANCE": ["min", "max", "size"],
    "STATUS_NUM": ["mean", "max"]
})

bb_agg.columns = [f"BB_{c[0]}_{c[1].upper()}" for c in bb_agg.columns]
bb_agg = bb_agg.reset_index()


In [ ]:
bb_temp = bureau_df[["SK_ID_BUREAU", "SK_ID_CURR"]].merge(bb_agg, on="SK_ID_BUREAU", how="left")


bb_agg_dict = {
    "BB_MONTHS_BALANCE_MIN": ["min", "max"],
    "BB_MONTHS_BALANCE_MAX": ["min", "max"],
    "BB_MONTHS_BALANCE_SIZE": ["sum", "max"],
    "BB_STATUS_NUM_MEAN": ["mean", "max"],
    "BB_STATUS_NUM_MAX": ["max"],
}

bb_final = bb_temp.groupby("SK_ID_CURR").agg(bb_agg_dict)

# flatten column names
bb_final.columns = ["_".join([col[0], col[1].upper()]) for col in bb_final.columns]
bb_final = bb_final.reset_index()

bb_final.head()

,SK_ID_CURR,BB_MONTHS_BALANCE_MIN_MIN,BB_MONTHS_BALANCE_MIN_MAX,BB_MONTHS_BALANCE_MAX_MIN,BB_MONTHS_BALANCE_MAX_MAX,BB_MONTHS_BALANCE_SIZE_SUM,BB_MONTHS_BALANCE_SIZE_MAX,BB_STATUS_NUM_MEAN_MEAN,BB_STATUS_NUM_MEAN_MAX,BB_STATUS_NUM_MAX_MAX
0,100001,-51.0,-1.0,0.0,0.0,172.0,52.0,0.229627,0.500000,2.0
1,100002,-47.0,-3.0,-32.0,0.0,110.0,22.0,0.673295,1.000000,2.0
2,100003,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,100004,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,100005,-12.0,-2.0,0.0,0.0,21.0,13.0,0.136752,0.333333,1.0


In [37]:
bureau_full = bureau_df.copy()

# 3. FEATURE ENGINEERING

In [ ]:
# Total of bureau loans for creating ratio
total_loans = bureau_df.groupby("SK_ID_CURR").size().rename("TOTAL_BUREAU_LOANS")

# Ordinal encode CREDIT_ACTIVE theo severity
credit_active_map = {
    "Closed": 0,
    "Active": 2,
    "Sold": 3,
    "Bad debt": 4  
}
bureau_full["CREDIT_ACTIVE_ENC"] = bureau_full["CREDIT_ACTIVE"].map(credit_active_map)

# Credit_active ratio
active_cnt = bureau_full[bureau_full["CREDIT_ACTIVE"]=="Active"].groupby("SK_ID_CURR").size().rename("CNT_ACTIVE")
sold_cnt   = bureau_full[bureau_full["CREDIT_ACTIVE"]=="Sold"].groupby("SK_ID_CURR").size().rename("CNT_SOLD")


high_risk = ((active_cnt>0) & (sold_cnt>0)).astype(int).rename("HIGH_CREDIT_RISK_FLAG")

# Currency flag
bureau_full["CURRENCY_1_FLAG"] = (bureau_full["CREDIT_CURRENCY"]=="currency 1").astype(int)

# Loan types
major_types = ["Consumer credit","Credit card","Car loan","Mortgage","Microloan"]

bureau_full["CREDIT_TYPE_GRP"] = bureau_full["CREDIT_TYPE"].apply(
    lambda x: x if x in major_types else "Other"
)


credit_type_map = {
    "Mortgage": 1,
    "Car loan": 0,
    "Consumer credit": 2,
    "Credit card": 3,
    "Microloan": 4,
    "Other": 2
}
bureau_full["CREDIT_TYPE_ENC"] = bureau_full["CREDIT_TYPE_GRP"].map(credit_type_map)

In [39]:
# Count overdue months
dpd1 = bureau_bl[bureau_bl["STATUS_NUM"] >= 1].groupby("SK_ID_BUREAU").size().rename("DPD1_COUNT")
dpd2 = bureau_bl[bureau_bl["STATUS_NUM"] >= 2].groupby("SK_ID_BUREAU").size().rename("DPD2_COUNT")
dpd3 = bureau_bl[bureau_bl["STATUS_NUM"] >= 3].groupby("SK_ID_BUREAU").size().rename("DPD3_COUNT")


last_status = (
    bureau_bl.sort_values(["SK_ID_BUREAU","MONTHS_BALANCE"])
             .groupby("SK_ID_BUREAU")["STATUS_NUM"]
             .last()
             .rename("LAST_STATUS")
)


overdue_mask = bureau_bl[bureau_bl["STATUS_NUM"]>=1]

days_last_overdue = (
    overdue_mask.groupby("SK_ID_BUREAU")["MONTHS_BALANCE"]
                .min()
                .rename("MONTHS_SINCE_LAST_OVERDUE")
)

status_features = pd.concat([dpd1, dpd2, dpd3, last_status, days_last_overdue], axis=1)
bureau_full = bureau_full.merge(status_features, on="SK_ID_BUREAU", how="left")

In [ ]:
def time_to_end_bucket(x):
    if pd.isna(x): return "unknown"
    if x <= 0: return "closed"
    if x <= 30: return "<=30d"
    if x <= 180: return "<=6m"
    if x <= 365: return "<=12m"
    return ">12m"

def safe_div(a, b):
    return a / b.mask(b == 0)

bureau_full["BUREAU_TIME_TO_END"] = bureau_full["DAYS_CREDIT_ENDDATE"].apply(time_to_end_bucket)
bureau_full = pd.get_dummies(bureau_full, columns=["BUREAU_TIME_TO_END"], prefix="TIMEEND")

In [ ]:
# DPD trend feature
def add_dpd_features_for_bureau(df):
    """
    Create DPD trend and recent-window features for one SK_BUREAU_ID.
    """
    # sort by time: older first, newest last (MONTHS_BALANCE is negative)
    df = df.sort_values("MONTHS_BALANCE")
    months = df["MONTHS_BALANCE"].values
    status = df["STATUS_NUM"].fillna(0).values

    # DPD trend
    if len(months) >= 2:
        slope = np.polyfit(months, status, 1)[0]
    else:
        slope = 0.0

    # Last 1 / 3 / 6 months mean DPD
    if len(status) >= 1:
        last1 = status[-1]
    else:
        last1 = 0.0

    if len(status) >= 1:
        last3 = status[-3:].mean()
        last6 = status[-6:].mean()
    else:
        last3 = 0.0
        last6 = 0.0

    # Transition features
    late = status > 0
    if len(late) >= 2:
        trans_0_to_late = ((late[:-1] == False) & (late[1:] == True)).sum()
        trans_late_to_0 = ((late[:-1] == True) & (late[1:] == False)).sum()
    else:
        trans_0_to_late = 0
        trans_late_to_0 = 0

    # longest streak of consecutive late months
    max_streak = 0
    current = 0
    for v in late:
        if v:
            current += 1
            if current > max_streak:
                max_streak = current
        else:
            current = 0

    return pd.Series({
        "BB_DPD_TREND": slope,
        "BB_DPD_LAST1M": last1,
        "BB_DPD_LAST3M": last3,
        "BB_DPD_LAST6M": last6,
        "BB_DPD_TRANS_0_TO_LATE": trans_0_to_late,
        "BB_DPD_TRANS_LATE_TO_0": trans_late_to_0,
        "BB_DPD_MAX_STREAK_LATE": max_streak
    })


bb_extra = bureau_bl.groupby("SK_BUREAU_ID").apply(add_dpd_features_for_bureau).reset_index()


bb_extra = bb_extra.merge(
    bureau_df[["SK_ID_BUREAU", "SK_ID_CURR"]],
    on="SK_ID_BUREAU",
    how="left"
)

bb_extra_agg = bb_extra.groupby("SK_ID_CURR").agg({
    "BB_DPD_TREND": ["mean", "max", "min"],
    "BB_DPD_LAST1M": ["mean", "max"],
    "BB_DPD_LAST3M": ["mean", "max"],
    "BB_DPD_LAST6M": ["mean", "max"],
    "BB_DPD_TRANS_0_TO_LATE": ["sum", "mean"],
    "BB_DPD_TRANS_LATE_TO_0": ["sum", "mean"],
    "BB_DPD_MAX_STREAK_LATE": ["max", "mean"]
}).reset_index()

# flatten columns
bb_extra_agg.columns = ["SK_ID_CURR"] + [
    "BB_EXTRA_" + "_".join(col).upper()
    for col in bb_extra_agg.columns[1:]
]


# Loan age features
bureau_full["LOAN_AGE_DAYS"] = -bureau_full["DAYS_CREDIT"]
bureau_full["LOAN_AGE_YEARS"] = bureau_full["LOAN_AGE_DAYS"] / 365.0

# Remaining duration and total duration (approx)
bureau_full["LOAN_REMAINING_DAYS"] = bureau_full["DAYS_CREDIT_ENDDATE"]
bureau_full["LOAN_DURATION_DAYS"] = bureau_full["DAYS_CREDIT_ENDDATE"] - bureau_full["DAYS_CREDIT"]
bureau_full["LOAN_DURATION_YEARS"] = bureau_full["LOAN_DURATION_DAYS"] / 365.0

bureau_full["LOAN_AGE_RATIO"] = safe_div(
    bureau_full["LOAN_AGE_DAYS"],
    bureau_full["LOAN_DURATION_DAYS"]
)

# Aggregate loan age features to client level
loan_age_agg = bureau_full.groupby("SK_ID_CURR")[[
    "LOAN_AGE_DAYS",
    "LOAN_AGE_YEARS",
    "LOAN_REMAINING_DAYS",
    "LOAN_DURATION_DAYS",
    "LOAN_DURATION_YEARS",
    "LOAN_AGE_RATIO"
]].agg(["mean", "max", "min"]).reset_index()

loan_age_agg.columns = ["SK_ID_CURR"] + [
    "BURO_LOAN_" + "_".join(col).upper()
    for col in loan_age_agg.columns[1:]
]



# Active loans features
active_df = bureau_full[bureau_full["CREDIT_ACTIVE"] == "Active"].copy()

active_agg = active_df.groupby("SK_ID_CURR").agg({
    "SK_ID_BUREAU": "count",       # number of active loans
    "DEBT_TO_CREDIT": "mean",
    "OVERDUE_RATIO": "mean",
    "DPD1_COUNT": "sum",
    "DPD2_COUNT": "sum",
    "DPD3_COUNT": "sum"
}).reset_index()

active_agg = active_agg.rename(columns={
    "SK_ID_BUREAU": "ACTIVE_LOANS",
    "DEBT_TO_CREDIT": "ACTIVE_DEBT_TO_CREDIT_MEAN",
    "OVERDUE_RATIO": "ACTIVE_OVERDUE_RATIO_MEAN",
    "DPD1_COUNT": "ACTIVE_DPD1_SUM",
    "DPD2_COUNT": "ACTIVE_DPD2_SUM",
    "DPD3_COUNT": "ACTIVE_DPD3_SUM"
})

# active loans / total bureau loans
active_agg = active_agg.merge(
    total_loans.reset_index(),
    on="SK_ID_CURR",
    how="left"
)
active_agg["ACTIVE_LOAN_RATIO"] = safe_div(
    active_agg["ACTIVE_LOANS"],
    active_agg["TOTAL_BUREAU_LOANS"]
)

In [ ]:
# Ratio features
bureau_full["PCT_CREDIT_LIMIT_USED"] = safe_div(bureau_full["AMT_CREDIT_SUM"], bureau_full["AMT_CREDIT_SUM_LIMIT"])
bureau_full["PCT_CREDIT_DEBT"] = safe_div(bureau_full["AMT_CREDIT_SUM_DEBT"], bureau_full["AMT_CREDIT_SUM_LIMIT"])
bureau_full["OVERDUE_RATIO"] = safe_div(bureau_full["AMT_CREDIT_SUM_OVERDUE"], bureau_full["AMT_CREDIT_SUM"])
bureau_full["DEBT_TO_CREDIT"] = safe_div(bureau_full["AMT_CREDIT_SUM_DEBT"], bureau_full["AMT_CREDIT_SUM"])
bureau_full["OVERDUE_DEBT"] = safe_div(bureau_full["AMT_CREDIT_SUM_OVERDUE"], bureau_full["AMT_CREDIT_SUM_DEBT"])
bureau_full["ANNUITY_TO_SUM"] = safe_div(bureau_full["AMT_ANNUITY"], bureau_full["AMT_CREDIT_SUM"])
bureau_full["ANNUITY_TO_DEBT"] = safe_div(bureau_full["AMT_ANNUITY"], bureau_full["AMT_CREDIT_SUM_DEBT"])


numeric_cols = bureau_full.select_dtypes(include=['number']).columns
bureau_numeric = bureau_full[numeric_cols].copy()
bureau_numeric["SK_ID_CURR"] = bureau_full["SK_ID_CURR"]

agg_functions = ["mean", "max", "min", "sum", "median"]

bureau_agg = bureau_numeric.groupby("SK_ID_CURR").agg(agg_functions)
bureau_agg.columns = ["BURO_" + "_".join(col).upper() for col in bureau_agg.columns]
bureau_agg = bureau_agg.reset_index()

In [42]:
bureau_final = bureau_agg.merge(bb_final, on="SK_ID_CURR", how="left")

In [43]:
bureau_final.shape

(305811, 145)

In [44]:
def check_nan(col):
    num_nan = col.isnull().sum()
    num_per = num_nan/len(col) * 100
    col_miss = pd.concat([num_nan, num_per], axis=1, keys=['number_of_NaN', 'percentage_of_NaN']).sort_values(by='percentage_of_NaN', ascending = False).round(1)
    col_miss = col_miss[col_miss['number_of_NaN'] != 0]
    print('Num fields: ', col.shape[1])
    print('Num missing fields: ', col_miss.shape[0])
    return col_miss

In [45]:
bureau_final.columns.tolist()

['SK_ID_CURR',
 'BURO_SK_ID_BUREAU_MEAN',
 'BURO_SK_ID_BUREAU_MAX',
 'BURO_SK_ID_BUREAU_MIN',
 'BURO_SK_ID_BUREAU_SUM',
 'BURO_SK_ID_BUREAU_MEDIAN',
 'BURO_DAYS_CREDIT_MEAN',
 'BURO_DAYS_CREDIT_MAX',
 'BURO_DAYS_CREDIT_MIN',
 'BURO_DAYS_CREDIT_SUM',
 'BURO_DAYS_CREDIT_MEDIAN',
 'BURO_CREDIT_DAY_OVERDUE_MEAN',
 'BURO_CREDIT_DAY_OVERDUE_MAX',
 'BURO_CREDIT_DAY_OVERDUE_MIN',
 'BURO_CREDIT_DAY_OVERDUE_SUM',
 'BURO_CREDIT_DAY_OVERDUE_MEDIAN',
 'BURO_DAYS_CREDIT_ENDDATE_MEAN',
 'BURO_DAYS_CREDIT_ENDDATE_MAX',
 'BURO_DAYS_CREDIT_ENDDATE_MIN',
 'BURO_DAYS_CREDIT_ENDDATE_SUM',
 'BURO_DAYS_CREDIT_ENDDATE_MEDIAN',
 'BURO_DAYS_ENDDATE_FACT_MEAN',
 'BURO_DAYS_ENDDATE_FACT_MAX',
 'BURO_DAYS_ENDDATE_FACT_MIN',
 'BURO_DAYS_ENDDATE_FACT_SUM',
 'BURO_DAYS_ENDDATE_FACT_MEDIAN',
 'BURO_CNT_CREDIT_PROLONG_MEAN',
 'BURO_CNT_CREDIT_PROLONG_MAX',
 'BURO_CNT_CREDIT_PROLONG_MIN',
 'BURO_CNT_CREDIT_PROLONG_SUM',
 'BURO_CNT_CREDIT_PROLONG_MEDIAN',
 'BURO_AMT_CREDIT_SUM_MEAN',
 'BURO_AMT_CREDIT_SUM_MAX',
 'BURO_AM

In [46]:
check_nan(bureau_final)

Num fields:  145
Num missing fields:  84


,number_of_NaN,percentage_of_NaN
BURO_DPD3_COUNT_MEDIAN,296485,97.0
BURO_DPD3_COUNT_MAX,296485,97.0
BURO_DPD3_COUNT_MIN,296485,97.0
BURO_DPD3_COUNT_MEAN,296485,97.0
BURO_DPD2_COUNT_MEAN,257289,84.1
BURO_DPD2_COUNT_MAX,257289,84.1
BURO_DPD2_COUNT_MIN,257289,84.1
BURO_DPD2_COUNT_MEDIAN,257289,84.1
BURO_PCT_CREDIT_DEBT_MEDIAN,249378,81.5
BURO_PCT_CREDIT_DEBT_MEAN,249378,81.5


In [ ]:
# Fill ratio logic
ratio_cols = [col for col in bureau_final.columns 
              if 'DPD' in col or 'RATIO' in col or 'DEBT' in col or 'PCT' in col]

bureau_final[ratio_cols] = bureau_final[ratio_cols].fillna(0)

# Fill numeric with median
num_cols = bureau_final.select_dtypes(include='number').columns
num_cols = [c for c in num_cols if c not in ratio_cols]

bureau_final[num_cols] = bureau_final[num_cols].fillna(bureau_final[num_cols].median())

# Fill one-hot / category flags
cat_cols = bureau_final.select_dtypes(include=['uint8']).columns
bureau_final[cat_cols] = bureau_final[cat_cols].fillna(0)

In [48]:
check_nan(bureau_final)

Num fields:  145
Num missing fields:  0


,number_of_NaN,percentage_of_NaN


In [49]:
bureau_application_train_feature = application_train.merge(bureau_final, on="SK_ID_CURR", how="left")
bureau_application_test_feature  = application_test.merge(bureau_final, on="SK_ID_CURR", how="left")

In [50]:
print(f'Shape of bureau_train_fe: {bureau_application_train_feature.shape}')
print(f'Shape of bureau_test_fe: {bureau_application_test_feature.shape}')

Shape of bureau_train_fe: (307511, 146)
Shape of bureau_test_fe: (48744, 145)


In [51]:
bureau_application_train_feature.head()

,SK_ID_CURR,TARGET,BURO_SK_ID_BUREAU_MEAN,BURO_SK_ID_BUREAU_MAX,BURO_SK_ID_BUREAU_MIN,BURO_SK_ID_BUREAU_SUM,BURO_SK_ID_BUREAU_MEDIAN,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_SUM,BURO_DAYS_CREDIT_MEDIAN,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MIN,BURO_CREDIT_DAY_OVERDUE_SUM,BURO_CREDIT_DAY_OVERDUE_MEDIAN,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_SUM,BURO_DAYS_CREDIT_ENDDATE_MEDIAN,BURO_DAYS_ENDDATE_FACT_MEAN,BURO_DAYS_ENDDATE_FACT_MAX,BURO_DAYS_ENDDATE_FACT_MIN,BURO_DAYS_ENDDATE_FACT_SUM,BURO_DAYS_ENDDATE_FACT_MEDIAN,BURO_CNT_CREDIT_PROLONG_MEAN,BURO_CNT_CREDIT_PROLONG_MAX,BURO_CNT_CREDIT_PROLONG_MIN,BURO_CNT_CREDIT_PROLONG_SUM,BURO_CNT_CREDIT_PROLONG_MEDIAN,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_MAX,BURO_AMT_CREDIT_SUM_MIN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_MEDIAN,BURO_AMT_CREDIT_SUM_DEBT_MEAN,BURO_AMT_CREDIT_SUM_DEBT_MAX,BURO_AMT_CREDIT_SUM_DEBT_MIN,BURO_AMT_CREDIT_SUM_DEBT_SUM,BURO_AMT_CREDIT_SUM_DEBT_MEDIAN,BURO_AMT_CREDIT_SUM_LIMIT_MEAN,BURO_AMT_CREDIT_SUM_LIMIT_MAX,BURO_AMT_CREDIT_SUM_LIMIT_MIN,BURO_AMT_CREDIT_SUM_LIMIT_SUM,BURO_AMT_CREDIT_SUM_LIMIT_MEDIAN,BURO_AMT_CREDIT_SUM_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_OVERDUE_MAX,BURO_AMT_CREDIT_SUM_OVERDUE_MIN,BURO_AMT_CREDIT_SUM_OVERDUE_SUM,BURO_AMT_CREDIT_SUM_OVERDUE_MEDIAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MAX,BURO_DAYS_CREDIT_UPDATE_MIN,BURO_DAYS_CREDIT_UPDATE_SUM,BURO_DAYS_CREDIT_UPDATE_MEDIAN,BURO_AMT_ANNUITY_MEAN,BURO_AMT_ANNUITY_MAX,BURO_AMT_ANNUITY_MIN,BURO_AMT_ANNUITY_SUM,BURO_AMT_ANNUITY_MEDIAN,BURO_CREDIT_ACTIVE_ENC_MEAN,BURO_CREDIT_ACTIVE_ENC_MAX,BURO_CREDIT_ACTIVE_ENC_MIN,BURO_CREDIT_ACTIVE_ENC_SUM,BURO_CREDIT_ACTIVE_ENC_MEDIAN,BURO_CURRENCY_1_FLAG_MEAN,BURO_CURRENCY_1_FLAG_MAX,BURO_CURRENCY_1_FLAG_MIN,BURO_CURRENCY_1_FLAG_SUM,BURO_CURRENCY_1_FLAG_MEDIAN,BURO_CREDIT_TYPE_ENC_MEAN,BURO_CREDIT_TYPE_ENC_MAX,BURO_CREDIT_TYPE_ENC_MIN,BURO_CREDIT_TYPE_ENC_SUM,BURO_CREDIT_TYPE_ENC_MEDIAN,BURO_DPD1_COUNT_MEAN,BURO_DPD1_COUNT_MAX,BURO_DPD1_COUNT_MIN,BURO_DPD1_COUNT_SUM,BURO_DPD1_COUNT_MEDIAN,BURO_DPD2_COUNT_MEAN,BURO_DPD2_COUNT_MAX,BURO_DPD2_COUNT_MIN,BURO_DPD2_COUNT_SUM,BURO_DPD2_COUNT_MEDIAN,BURO_DPD3_COUNT_MEAN,BURO_DPD3_COUNT_MAX,BURO_DPD3_COUNT_MIN,BURO_DPD3_COUNT_SUM,BURO_DPD3_COUNT_MEDIAN,BURO_LAST_STATUS_MEAN,BURO_LAST_STATUS_MAX,BURO_LAST_STATUS_MIN,BURO_LAST_STATUS_SUM,BURO_LAST_STATUS_MEDIAN,BURO_MONTHS_SINCE_LAST_OVERDUE_MEAN,BURO_MONTHS_SINCE_LAST_OVERDUE_MAX,BURO_MONTHS_SINCE_LAST_OVERDUE_MIN,BURO_MONTHS_SINCE_LAST_OVERDUE_SUM,BURO_MONTHS_SINCE_LAST_OVERDUE_MEDIAN,BURO_PCT_CREDIT_LIMIT_USED_MEAN,BURO_PCT_CREDIT_LIMIT_USED_MAX,BURO_PCT_CREDIT_LIMIT_USED_MIN,BURO_PCT_CREDIT_LIMIT_USED_SUM,BURO_PCT_CREDIT_LIMIT_USED_MEDIAN,BURO_PCT_CREDIT_DEBT_MEAN,BURO_PCT_CREDIT_DEBT_MAX,BURO_PCT_CREDIT_DEBT_MIN,BURO_PCT_CREDIT_DEBT_SUM,BURO_PCT_CREDIT_DEBT_MEDIAN,BURO_OVERDUE_RATIO_MEAN,BURO_OVERDUE_RATIO_MAX,BURO_OVERDUE_RATIO_MIN,BURO_OVERDUE_RATIO_SUM,BURO_OVERDUE_RATIO_MEDIAN,BURO_DEBT_TO_CREDIT_MEAN,BURO_DEBT_TO_CREDIT_MAX,BURO_DEBT_TO_CREDIT_MIN,BURO_DEBT_TO_CREDIT_SUM,BURO_DEBT_TO_CREDIT_MEDIAN,BURO_OVERDUE_DEBT_MEAN,BURO_OVERDUE_DEBT_MAX,BURO_OVERDUE_DEBT_MIN,BURO_OVERDUE_DEBT_SUM,BURO_OVERDUE_DEBT_MEDIAN,BURO_ANNUITY_TO_SUM_MEAN,BURO_ANNUITY_TO_SUM_MAX,BURO_ANNUITY_TO_SUM_MIN,BURO_ANNUITY_TO_SUM_SUM,BURO_ANNUITY_TO_SUM_MEDIAN,BURO_ANNUITY_TO_DEBT_MEAN,BURO_ANNUITY_TO_DEBT_MAX,BURO_ANNUITY_TO_DEBT_MIN,BURO_ANNUITY_TO_DEBT_SUM,BURO_ANNUITY_TO_DEBT_MEDIAN,BB_MONTHS_BALANCE_MIN_MIN,BB_MONTHS_BALANCE_MIN_MAX,BB_MONTHS_BALANCE_MAX_MIN,BB_MONTHS_BALANCE_MAX_MAX,BB_MONTHS_BALANCE_SIZE_SUM,BB_MONTHS_BALANCE_SIZE_MAX,BB_STATUS_NUM_MEAN_MEAN,BB_STATUS_NUM_MEAN_MAX,BB_STATUS_NUM_MAX_MAX
0,100002,1,6153272.125,6158909.0,6113835.0,49226177.0,6158905.5,-874.00,-103.0,-1437.0,-6992.0,-1042.5,0.0,0.0,0.0,0.0,0.0,-349.0,780.0,-1072.0,-2094.0,-424.5,-697.500000,-36.0,-1185.0,-4185.0,-939.0,0.0,0.0,0.0,0.0,

In [52]:
bureau_application_train_feature.columns.to_list()

['SK_ID_CURR',
 'TARGET',
 'BURO_SK_ID_BUREAU_MEAN',
 'BURO_SK_ID_BUREAU_MAX',
 'BURO_SK_ID_BUREAU_MIN',
 'BURO_SK_ID_BUREAU_SUM',
 'BURO_SK_ID_BUREAU_MEDIAN',
 'BURO_DAYS_CREDIT_MEAN',
 'BURO_DAYS_CREDIT_MAX',
 'BURO_DAYS_CREDIT_MIN',
 'BURO_DAYS_CREDIT_SUM',
 'BURO_DAYS_CREDIT_MEDIAN',
 'BURO_CREDIT_DAY_OVERDUE_MEAN',
 'BURO_CREDIT_DAY_OVERDUE_MAX',
 'BURO_CREDIT_DAY_OVERDUE_MIN',
 'BURO_CREDIT_DAY_OVERDUE_SUM',
 'BURO_CREDIT_DAY_OVERDUE_MEDIAN',
 'BURO_DAYS_CREDIT_ENDDATE_MEAN',
 'BURO_DAYS_CREDIT_ENDDATE_MAX',
 'BURO_DAYS_CREDIT_ENDDATE_MIN',
 'BURO_DAYS_CREDIT_ENDDATE_SUM',
 'BURO_DAYS_CREDIT_ENDDATE_MEDIAN',
 'BURO_DAYS_ENDDATE_FACT_MEAN',
 'BURO_DAYS_ENDDATE_FACT_MAX',
 'BURO_DAYS_ENDDATE_FACT_MIN',
 'BURO_DAYS_ENDDATE_FACT_SUM',
 'BURO_DAYS_ENDDATE_FACT_MEDIAN',
 'BURO_CNT_CREDIT_PROLONG_MEAN',
 'BURO_CNT_CREDIT_PROLONG_MAX',
 'BURO_CNT_CREDIT_PROLONG_MIN',
 'BURO_CNT_CREDIT_PROLONG_SUM',
 'BURO_CNT_CREDIT_PROLONG_MEDIAN',
 'BURO_AMT_CREDIT_SUM_MEAN',
 'BURO_AMT_CREDIT_SUM_MAX'

In [53]:
bureau_application_test_feature.head()

,SK_ID_CURR,BURO_SK_ID_BUREAU_MEAN,BURO_SK_ID_BUREAU_MAX,BURO_SK_ID_BUREAU_MIN,BURO_SK_ID_BUREAU_SUM,BURO_SK_ID_BUREAU_MEDIAN,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_SUM,BURO_DAYS_CREDIT_MEDIAN,BURO_CREDIT_DAY_OVERDUE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MIN,BURO_CREDIT_DAY_OVERDUE_SUM,BURO_CREDIT_DAY_OVERDUE_MEDIAN,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_SUM,BURO_DAYS_CREDIT_ENDDATE_MEDIAN,BURO_DAYS_ENDDATE_FACT_MEAN,BURO_DAYS_ENDDATE_FACT_MAX,BURO_DAYS_ENDDATE_FACT_MIN,BURO_DAYS_ENDDATE_FACT_SUM,BURO_DAYS_ENDDATE_FACT_MEDIAN,BURO_CNT_CREDIT_PROLONG_MEAN,BURO_CNT_CREDIT_PROLONG_MAX,BURO_CNT_CREDIT_PROLONG_MIN,BURO_CNT_CREDIT_PROLONG_SUM,BURO_CNT_CREDIT_PROLONG_MEDIAN,BURO_AMT_CREDIT_SUM_MEAN,BURO_AMT_CREDIT_SUM_MAX,BURO_AMT_CREDIT_SUM_MIN,BURO_AMT_CREDIT_SUM_SUM,BURO_AMT_CREDIT_SUM_MEDIAN,BURO_AMT_CREDIT_SUM_DEBT_MEAN,BURO_AMT_CREDIT_SUM_DEBT_MAX,BURO_AMT_CREDIT_SUM_DEBT_MIN,BURO_AMT_CREDIT_SUM_DEBT_SUM,BURO_AMT_CREDIT_SUM_DEBT_MEDIAN,BURO_AMT_CREDIT_SUM_LIMIT_MEAN,BURO_AMT_CREDIT_SUM_LIMIT_MAX,BURO_AMT_CREDIT_SUM_LIMIT_MIN,BURO_AMT_CREDIT_SUM_LIMIT_SUM,BURO_AMT_CREDIT_SUM_LIMIT_MEDIAN,BURO_AMT_CREDIT_SUM_OVERDUE_MEAN,BURO_AMT_CREDIT_SUM_OVERDUE_MAX,BURO_AMT_CREDIT_SUM_OVERDUE_MIN,BURO_AMT_CREDIT_SUM_OVERDUE_SUM,BURO_AMT_CREDIT_SUM_OVERDUE_MEDIAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MAX,BURO_DAYS_CREDIT_UPDATE_MIN,BURO_DAYS_CREDIT_UPDATE_SUM,BURO_DAYS_CREDIT_UPDATE_MEDIAN,BURO_AMT_ANNUITY_MEAN,BURO_AMT_ANNUITY_MAX,BURO_AMT_ANNUITY_MIN,BURO_AMT_ANNUITY_SUM,BURO_AMT_ANNUITY_MEDIAN,BURO_CREDIT_ACTIVE_ENC_MEAN,BURO_CREDIT_ACTIVE_ENC_MAX,BURO_CREDIT_ACTIVE_ENC_MIN,BURO_CREDIT_ACTIVE_ENC_SUM,BURO_CREDIT_ACTIVE_ENC_MEDIAN,BURO_CURRENCY_1_FLAG_MEAN,BURO_CURRENCY_1_FLAG_MAX,BURO_CURRENCY_1_FLAG_MIN,BURO_CURRENCY_1_FLAG_SUM,BURO_CURRENCY_1_FLAG_MEDIAN,BURO_CREDIT_TYPE_ENC_MEAN,BURO_CREDIT_TYPE_ENC_MAX,BURO_CREDIT_TYPE_ENC_MIN,BURO_CREDIT_TYPE_ENC_SUM,BURO_CREDIT_TYPE_ENC_MEDIAN,BURO_DPD1_COUNT_MEAN,BURO_DPD1_COUNT_MAX,BURO_DPD1_COUNT_MIN,BURO_DPD1_COUNT_SUM,BURO_DPD1_COUNT_MEDIAN,BURO_DPD2_COUNT_MEAN,BURO_DPD2_COUNT_MAX,BURO_DPD2_COUNT_MIN,BURO_DPD2_COUNT_SUM,BURO_DPD2_COUNT_MEDIAN,BURO_DPD3_COUNT_MEAN,BURO_DPD3_COUNT_MAX,BURO_DPD3_COUNT_MIN,BURO_DPD3_COUNT_SUM,BURO_DPD3_COUNT_MEDIAN,BURO_LAST_STATUS_MEAN,BURO_LAST_STATUS_MAX,BURO_LAST_STATUS_MIN,BURO_LAST_STATUS_SUM,BURO_LAST_STATUS_MEDIAN,BURO_MONTHS_SINCE_LAST_OVERDUE_MEAN,BURO_MONTHS_SINCE_LAST_OVERDUE_MAX,BURO_MONTHS_SINCE_LAST_OVERDUE_MIN,BURO_MONTHS_SINCE_LAST_OVERDUE_SUM,BURO_MONTHS_SINCE_LAST_OVERDUE_MEDIAN,BURO_PCT_CREDIT_LIMIT_USED_MEAN,BURO_PCT_CREDIT_LIMIT_USED_MAX,BURO_PCT_CREDIT_LIMIT_USED_MIN,BURO_PCT_CREDIT_LIMIT_USED_SUM,BURO_PCT_CREDIT_LIMIT_USED_MEDIAN,BURO_PCT_CREDIT_DEBT_MEAN,BURO_PCT_CREDIT_DEBT_MAX,BURO_PCT_CREDIT_DEBT_MIN,BURO_PCT_CREDIT_DEBT_SUM,BURO_PCT_CREDIT_DEBT_MEDIAN,BURO_OVERDUE_RATIO_MEAN,BURO_OVERDUE_RATIO_MAX,BURO_OVERDUE_RATIO_MIN,BURO_OVERDUE_RATIO_SUM,BURO_OVERDUE_RATIO_MEDIAN,BURO_DEBT_TO_CREDIT_MEAN,BURO_DEBT_TO_CREDIT_MAX,BURO_DEBT_TO_CREDIT_MIN,BURO_DEBT_TO_CREDIT_SUM,BURO_DEBT_TO_CREDIT_MEDIAN,BURO_OVERDUE_DEBT_MEAN,BURO_OVERDUE_DEBT_MAX,BURO_OVERDUE_DEBT_MIN,BURO_OVERDUE_DEBT_SUM,BURO_OVERDUE_DEBT_MEDIAN,BURO_ANNUITY_TO_SUM_MEAN,BURO_ANNUITY_TO_SUM_MAX,BURO_ANNUITY_TO_SUM_MIN,BURO_ANNUITY_TO_SUM_SUM,BURO_ANNUITY_TO_SUM_MEDIAN,BURO_ANNUITY_TO_DEBT_MEAN,BURO_ANNUITY_TO_DEBT_MAX,BURO_ANNUITY_TO_DEBT_MIN,BURO_ANNUITY_TO_DEBT_SUM,BURO_ANNUITY_TO_DEBT_MEDIAN,BB_MONTHS_BALANCE_MIN_MIN,BB_MONTHS_BALANCE_MIN_MAX,BB_MONTHS_BALANCE_MAX_MIN,BB_MONTHS_BALANCE_MAX_MAX,BB_MONTHS_BALANCE_SIZE_SUM,BB_MONTHS_BALANCE_SIZE_MAX,BB_STATUS_NUM_MEAN_MEAN,BB_STATUS_NUM_MEAN_MAX,BB_STATUS_NUM_MAX_MAX
0,100001,5896633.00,5896636.0,5896630.0,41276431.0,5896633.0,-735.000000,-49.0,-1572.0,-5145.0,-857.0,0.0,0.0,0.0,0.0,0.0,82.428571,1778.0,-1329.0,577.0,-179.0,-825.500000,-544.0,-1328.0,-3302.0,-715.0,0.0,0.0,0.0,0.0,0.0,2

In [54]:
bureau_application_train_feature.to_csv(r'../data/features/bureau_fe_train.csv', index=False)
bureau_application_test_feature.to_csv(r'../data/features/bureau_fe_test.csv', index=False)